## 1. Introduction/Business Problem

When you driving to another city, you may come across a terrible traffic jam on the high way, it may be an car accident due to bad weather such as rainy and windy. 
Now, wouldn't it be great if there is something in place that could warn you, given the weather and the road conditions about the possibility of you getting into a 
car accident and how severe it would be, so that you would drive more carefully or even change your travel if you are able to. 

The business problem is to predict severity of an accident using relative varables such as weather and road condition, based on prediction score we can set up the warning system.  


#

## 2. Data Understanding / Preparation

Provided data is collision data from all years by traffic records.
This includes all types of collisions. Collisions will display at the intersection or mid-block of a segment. Timeframe: 2004 to Present.

Data contains 38 columns and 194673 rows. 
check missing data and any adjusment. 
select relative columns of severity from columns and check the corelations among the variables 
review data relationship for severitycode. 

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [20]:
# CSV file tranfer 
!wget -o -- https://s3.us.cloud-object-storage.appdomain.cloud/cf-courses-data/CognitiveClass/DP0701EN/version-2/Data-Collisions.csv

In [21]:
# data review 
data = pd.read_csv('Data-Collisions.csv')
data.head()

/opt/conda/envs/Python36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (33) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,SEVERITYCODE,X,Y,OBJECTID,INCKEY,COLDETKEY,REPORTNO,STATUS,ADDRTYPE,INTKEY,...,ROADCOND,LIGHTCOND,PEDROWNOTGRNT,SDOTCOLNUM,SPEEDING,ST_COLCODE,ST_COLDESC,SEGLANEKEY,CROSSWALKKEY,HITPARKEDCAR
0,2,-122.323148,47.703140,1,1307,1307,3502005,Matched,Intersection,37475.0,...,Wet,Daylight,NaN,NaN,NaN,10,Entering at angle,0,0,N
1,1,-122.347294,47.647172,2,52200,52200,2607959,Matched,Block,NaN,...,Wet,Dark - Street Lights On,NaN,6354039.0,NaN,11,From same direction - both going straight - bo...,0,0,N
2,1,-122.334540,47.607871,3,26700,26700,1482393,Matched,Block,NaN,...,Dry,Daylight,NaN,4323031.0,NaN,32,One parked--one moving,0,0,N
3,1,-122.334803,47.604803,4,1144,1144,3503937,Matched,Block,NaN,...,Dry,Daylight,NaN,NaN,NaN,23,From same direction - all others,0,0,N
4,2,-122.306426,47.545739,5,17700,17700,1807429,Matched,Intersection,34387.0,...,Wet,Daylight,NaN,4028032.0,NaN,10,Entering at angle,0,0,N


In [23]:
data.shape

(194673, 38)

In [24]:
data.columns

Index(['SEVERITYCODE', 'X', 'Y', 'OBJECTID', 'INCKEY', 'COLDETKEY', 'REPORTNO',
       'STATUS', 'ADDRTYPE', 'INTKEY', 'LOCATION', 'EXCEPTRSNCODE',
       'EXCEPTRSNDESC', 'SEVERITYCODE.1', 'SEVERITYDESC', 'COLLISIONTYPE',
       'PERSONCOUNT', 'PEDCOUNT', 'PEDCYLCOUNT', 'VEHCOUNT', 'INCDATE',
       'INCDTTM', 'JUNCTIONTYPE', 'SDOT_COLCODE', 'SDOT_COLDESC',
       'INATTENTIONIND', 'UNDERINFL', 'WEATHER', 'ROADCOND', 'LIGHTCOND',
       'PEDROWNOTGRNT', 'SDOTCOLNUM', 'SPEEDING', 'ST_COLCODE', 'ST_COLDESC',
       'SEGLANEKEY', 'CROSSWALKKEY', 'HITPARKEDCAR'],
      dtype='object')

In [28]:
# extract relative data
df=data.loc[:,['SEVERITYCODE','ADDRTYPE','ROADCOND','LIGHTCOND','WEATHER']]
df.head()

,SEVERITYCODE,ADDRTYPE,ROADCOND,LIGHTCOND,WEATHER
0,2,Intersection,Wet,Daylight,Overcast
1,1,Block,Wet,Dark - Street Lights On,Raining
2,1,Block,Dry,Daylight,Overcast
3,1,Block,Dry,Daylight,Clear
4,2,Intersection,Wet,Daylight,Raining


In [ ]:
# Cleaning missing data 
# group by tables 
# check corelation of each items with severity code

In [29]:
df.dropna(axis=0, inplace=True)
df.head()

,SEVERITYCODE,ADDRTYPE,ROADCOND,LIGHTCOND,WEATHER
0,2,Intersection,Wet,Daylight,Overcast
1,1,Block,Wet,Dark - Street Lights On,Raining
2,1,Block,Dry,Daylight,Overcast
3,1,Block,Dry,Daylight,Clear
4,2,Intersection,Wet,Daylight,Raining


In [30]:
df.shape

(187525, 5)

In [75]:
df_sev = df.groupby(['SEVERITYCODE'], as_index=False).count()
df_sev

,SEVERITYCODE,ADDRTYPE,ROADCOND,LIGHTCOND,WEATHER
0,1,130642,130642,130642,130642
1,2,56883,56883,56883,56883


In [110]:
Add=df['WEATHER'].groupby([df['SEVERITYCODE'], df['ADDRTYPE']]).count()
Add.unstack()

ADDRTYPE,Alley,Block,Intersection
SEVERITYCODE,,,
1,662,93892,36088
2,80,29429,27374


In [111]:
Road=df['WEATHER'].groupby([df['SEVERITYCODE'], df['ROADCOND']]).count()
Road.unstack()

ROADCOND,Dry,Ice,Oil,Other,Sand/Mud/Dirt,Snow/Slush,Standing Water,Unknown,Wet
SEVERITYCODE,,,,,,,,,
1,83835,923,40,82,51,827,82,13279,31523
2,39901,270,24,42,22,165,29,730,15700


In [112]:
Light=df['WEATHER'].groupby([df['SEVERITYCODE'], df['LIGHTCOND']]).count()
Light.unstack()

LIGHTCOND,Dark - No Street Lights,Dark - Street Lights Off,Dark - Street Lights On,Dark - Unknown Lighting,Dawn,Daylight,Dusk,Other,Unknown
SEVERITYCODE,,,,,,,,,
1,1192,869,33816,7,1668,76998,3907,175,12010
2,334,315,14420,4,823,38410,1936,52,589


In [113]:
Wea=df['LIGHTCOND'].groupby([df['SEVERITYCODE'], df['WEATHER']]).count()
Wea.unstack()

WEATHER,Blowing Sand/Dirt,Clear,Fog/Smog/Smoke,Other,Overcast,Partly Cloudy,Raining,Severe Crosswind,Sleet/Hail/Freezing Rain,Snowing,Unknown
SEVERITYCODE,,,,,,,,,,,
1,36,74778,377,676,18836,2,21836,18,85,729,13269
2,13,35721,186,114,8715,3,11140,7,27,167,790
